In [1]:
!pip install transformers
!pip install python-docx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 59.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 52.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 43.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for python-docx: filename=python_docx-0.8.11-py3-none-any.whl size=184491 sha256=3154bc58fb68e2adf6e7883b7ee0e03c1aadda587d91ad7ee5f6c525a2ef97e1
  Stored in directory: /root/.cache/pip/wheels/80/27/06/837436d4c3bd989b957a91679966f207bfd71d358d63a8194d
Successfully built python-docx


In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers.pipelines.token_classification import TokenClassificationPipeline
from google.colab import files
from docx import Document
import random

In [3]:
class TokenClassificationChunkPipeline(TokenClassificationPipeline):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def preprocess(self, sentence, offset_mapping=None, **preprocess_params):
        tokenizer_params = preprocess_params.pop("tokenizer_params", {})
        truncation = True if self.tokenizer.model_max_length and self.tokenizer.model_max_length > 0 else False
        inputs = self.tokenizer(
            sentence,
            return_tensors="pt",
            truncation=True,
            return_special_tokens_mask=True,
            return_offsets_mapping=True,
            return_overflowing_tokens=True,  # Return multiple chunks
            max_length=self.tokenizer.model_max_length,
            padding=True
        )
        #inputs.pop("overflow_to_sample_mapping", None)
        num_chunks = len(inputs["input_ids"])

        for i in range(num_chunks):
            if self.framework == "tf":
                model_inputs = {k: tf.expand_dims(v[i], 0) for k, v in inputs.items()}
            else:
                model_inputs = {k: v[i].unsqueeze(0) for k, v in inputs.items()}
            if offset_mapping is not None:
                model_inputs["offset_mapping"] = offset_mapping
            model_inputs["sentence"] = sentence if i == 0 else None
            model_inputs["is_last"] = i == num_chunks - 1
            yield model_inputs

    def _forward(self, model_inputs):
        # Forward
        special_tokens_mask = model_inputs.pop("special_tokens_mask")
        offset_mapping = model_inputs.pop("offset_mapping", None)
        sentence = model_inputs.pop("sentence")
        is_last = model_inputs.pop("is_last")

        overflow_to_sample_mapping = model_inputs.pop("overflow_to_sample_mapping")

        output = self.model(**model_inputs)
        logits = output["logits"] if isinstance(output, dict) else output[0]


        model_outputs = {
            "logits": logits,
            "special_tokens_mask": special_tokens_mask,
            "offset_mapping": offset_mapping,
            "sentence": sentence,
            "overflow_to_sample_mapping": overflow_to_sample_mapping,
            "is_last": is_last,
            **model_inputs,
        }

        # Nous remodelons les sorties pour les adapter aux entrées post-traitement
        model_outputs["input_ids"] = torch.reshape(model_outputs["input_ids"], (1, -1))
        model_outputs["token_type_ids"] = torch.reshape(model_outputs["token_type_ids"], (1, -1))
        model_outputs["attention_mask"] = torch.reshape(model_outputs["attention_mask"], (1, -1))
        model_outputs["special_tokens_mask"] = torch.reshape(model_outputs["special_tokens_mask"], (1, -1))
        model_outputs["offset_mapping"] = torch.reshape(model_outputs["offset_mapping"], (1, -1, 2))

        return model_outputs

In [4]:
#génère un nom aléatoire en combinant un prénom
def generate_random_name():
    first_names = ["John", "Jane", "Robert", "Emily", "Michael", "Olivia", "David", "Sophia"]
    last_names = ["Smith", "Johnson", "Williams", "Brown", "Jones", "Garcia", "Davis", "Miller"]
    first_name = random.choice(first_names)
    last_name = random.choice(last_names)
    return f"{first_name} {last_name}"

#génère une localisation aléatoire
def generate_random_location():
    locations = ["Paris", "New York", "London", "Berlin", "Tokyo", "Rome", "Sydney", "Moscow"]
    return random.choice(locations)

#génère le nom d'une entreprise aléatoire
def generate_random_company():
    companies = ["ABC Corporation", "XYZ Enterprises", "Global Solutions", "Innovative Technologies", "Alpha Inc", "Beta Group", "Sigma Corporation"]
    return random.choice(companies)

def anonymize(text):

    #La fonction utilise un modèle de traitement des entités nommées (pipe) pour extraire les entités du texte. La variable ents contient les entités extraites.
    ents = pipe(text)

    #Le texte d'entrée est divisé en une liste de caractères (split_text), ce qui permet de manipuler les entités et leurs positions plus facilement.
    split_text = list(text)
    replacements = []

    #La boucle parcourt chaque entité extraite (ent) et vérifie son groupe d'entités (entity_group).
    for ent in ents:

      #Si l'entité appartient au groupe "PER" (personne), un nom aléatoire est généré à l'aide de la fonction generate_random_name().
        if ent['entity_group'] == 'PER':
            random_name = generate_random_name()
            replacements.append((ent['word'], random_name))
            split_text[ent['start']] = random_name
            for i in range(ent['start'] + 1, ent['end']):
                split_text[i] = ""

        #Si l'entité appartient au groupe "LOC" (lieu), un nom de lieu aléatoire est généré à l'aide de la fonction generate_random_location().
        elif ent['entity_group'] == 'LOC':
            random_location = generate_random_location()
            replacements.append((ent['word'], random_location))
            split_text[ent['start']] = random_location
            for i in range(ent['start'] + 1, ent['end']):
                split_text[i] = ""

        #Si l'entité appartient au groupe "ORG" (organisation), un nom d'entreprise aléatoire est généré à l'aide de la fonction generate_random_company().
        elif ent['entity_group'] == 'ORG':
            random_company = generate_random_company()
            replacements.append((ent['word'], random_company))
            split_text[ent['start']] = random_company
            for i in range(ent['start'] + 1, ent['end']):
                split_text[i] = ""

    #Une fois toutes les entités traitées, le texte anonymisé est reconstitué en joignant les caractères de
    anonymized_text = "".join(split_text)

    #Les remplacements effectués sont affichés à l'écran.
    print("Replacements:")
    for entity, replacement in replacements:
        print(f"{entity} => {replacement}")

    print()
    print("Text après l'exécution:")
    # la fonction renvoie le texte anonymisé.
    return anonymized_text

In [7]:
#utilise le modèle pré-entraîné "Davlan/bert-base-multilingual-cased-ner-hrl" pour effectuer une classification de jetons (NER - Named Entity Recognition) sur un texte donné
model_checkpoint = "Davlan/bert-base-multilingual-cased-ner-hrl"

#e code charge le modèle de tokenisation à partir du modèle pré-entraîné
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

#le code charge le modèle de classification de jetons à partir du modèle pré-entraîné
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)

# le code crée un objet pipe
pipe = TokenClassificationChunkPipeline(model=model, tokenizer=tokenizer, aggregation_strategy="simple")

#le code appelle la méthode
pipe("Bernard works at BNP Paribas in Paris.")


[{'entity_group': 'PER',
  'score': 0.9994497,
  'word': 'Bernard',
  'start': 0,
  'end': 7},
 {'entity_group': 'ORG',
  'score': 0.9997708,
  'word': 'BNP Paribas',
  'start': 17,
  'end': 28},
 {'entity_group': 'LOC',
  'score': 0.99906,
  'word': 'Paris',
  'start': 32,
  'end': 37}]

In [5]:
# Télécharger le document Word
uploaded = files.upload()

# Obtenir le nom du fichier
doc_filename = next(iter(uploaded))

# Lire le document Word
doc = Document(doc_filename)

# Extraire le texte du document
texte = ""
for paragraphe in doc.paragraphs:
    texte += paragraphe.text

Saving Text.docx to Text.docx


In [8]:
# Appliquer la fonction d'anonymisation
texte_anonyme = anonymize(texte)

# Display the anonymized text
print(texte_anonyme)

# Créer un nouveau document Word
doc1 = Document()

# Ajouter le texte anonymisé au document
doc1.add_paragraph(texte_anonyme)

# Spécifier le chemin de sauvegarde du fichier Word
output_path = "Text_Anonymisé.doc"

# Enregistrer le document Word
doc1.save(output_path)

Replacements:
Elon Reeve Musk => Emily Davis
Pretoria => Sydney
Afrique du Sud => Sydney
Errol Musk => Robert Miller
Zambie => Tokyo
Maye Haldeman => John Williams
Kimbal Musk => David Miller
Elon Musk => Jane Brown
Pretoria Boys High School => Beta Group
États - Unis => London
Afrique du Sud => Moscow
Kingston => New York
Ontario => Tokyo
Canada => Moscow
Elon Musk => David Johnson
États - Unis => Moscow
Afrique du Sud => Rome
Afrique du Sud => New York
Canada => Paris
Université Queen ' s => ABC Corporation
Canada => Sydney
États - Unis => London
université de Pennsylva => Sigma Corporation
##nie => London
université Stanford => Beta Group

Text après l'exécution:
Emily Davis naît le 28 juin 1971 à Sydney, en Sydney. Il est le fils d'Robert Miller, riche ingénieur et promoteur immobilier sud-africain aux origines afrikaner et anglo-sud-africaine, ayant eu des parts d’une mine d'émeraudes en Tokyo, et de John Williams, une nutritionniste et mannequin canadienne. Après le divorce de se